In [29]:
import pandas as pd
books_data = [("1984", "George Orwell", "Science Fiction", 5000, 1949),
          	  ("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
              ("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
 	            ("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
              ("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)]
books = pd.DataFrame(books_data, columns = ["title", "author", "genre", "sales", "year"])
books.index.name = 'id'
books.to_csv('/content/file.csv', sep=',')
books

,title,author,genre,sales,year
id,,,,,
0,1984,George Orwell,Science Fiction,5000,1949
1,The Lord of the Rings,J.R.R. Tolkien,Fantasy,3000,1954
2,To Kill a Mockingbird,Harper Lee,Southern Gothic,4000,1960
3,The Catcher in the Rye,J.D. Salinger,Novel,2000,1951
4,The Great Gatsby,F. Scott Fitzgerald,Novel,4500,1925


# Домашнее задание 5:
1. Используя Spark прочитайте данные из файла csv
2. Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
3. Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
4. Отсортируйте данные по общему объему продаж в порядке убывания.
5. Выведите результаты на экран.

In [30]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

conf = SparkConf().setAppName("DZ5")
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()

# Загружаем данные из csv файла и преобразуем типы данных
df = spark.read.csv("/content/file.csv", header=True, inferSchema=True)
df = df.withColumn("sales", df["sales"].cast('int'))
df = df.withColumn("year", df["year"].cast('int'))
# Фильтруем данные, , чтобы оставить только книги, продажи которых превышают 3000 экземпляров
df_filtered = df.filter(df.sales > 3000)

df_filtered.show()

+---+--------------------+-------------------+---------------+-----+----+
| id|               title|             author|          genre|sales|year|
+---+--------------------+-------------------+---------------+-----+----+
|  0|                1984|      George Orwell|Science Fiction| 5000|1949|
|  2|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|  4|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+---+--------------------+-------------------+---------------+-----+----+



In [33]:
#группируем неотфильтрованые данные по жанру и суммируем общий объем продаж по жанру в столбец с новым именем
df_genres = df.groupBy("genre").agg(sum("sales").alias("sum_sales"))

#вывод результата с сортировкой
df_genres.orderBy('sum_sales').show()

+---------------+---------+
|          genre|sum_sales|
+---------------+---------+
|        Fantasy|     3000|
|Southern Gothic|     4000|
|Science Fiction|     5000|
|          Novel|     6500|
+---------------+---------+



In [34]:
#выводим загруженные из файла данные отсортированные по продажам в порядке убывания
df_sorted = df.orderBy('sales', ascending=False).show()

+---+--------------------+-------------------+---------------+-----+----+
| id|               title|             author|          genre|sales|year|
+---+--------------------+-------------------+---------------+-----+----+
|  0|                1984|      George Orwell|Science Fiction| 5000|1949|
|  4|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|  2|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|  1|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|  3|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
+---+--------------------+-------------------+---------------+-----+----+



In [35]:
spark.stop()